# 07-02 Function Calling（工具调用）

让 AI 能够调用外部函数/工具，是构建智能体的核心能力。

## 1. 基础工具调用

In [ ]:
// 定义工具
const tools = [
  {
    type: 'function',
    function: {
      name: 'getWeather',
      description: '获取指定城市的天气',
      parameters: {
        type: 'object',
        properties: {
          city: {
            type: 'string',
            description: '城市名称'
          }
        },
        required: ['city']
      }
    }
  },
  {
    type: 'function',
    function: {
      name: 'sendMessage',
      description: '发送消息给用户',
      parameters: {
        type: 'object',
        properties: {
          to: { type: 'string' },
          content: { type: 'string' }
        },
        required: ['to', 'content']
      }
    }
  }
]

// 工具实现
const functions = {
  getWeather: ({ city }) => {
    return { city, temperature: 25, weather: 'sunny' };
  },
  sendMessage: ({ to, content }) => {
    console.log(`发送给 ${to}: ${content}`);
    return { success: true };
  }
};

## 2. 调用流程

In [ ]:
async function chatWithTools(userMessage) {
  const messages = [
    {
      role: 'system',
      content: '你是一个助手，可以使用提供的工具帮助用户'
    },
    { role: 'user', content: userMessage }
  ];
  
  // 第一次调用：AI 决定是否使用工具
  const response = await openai.chat.completions.create({
    model: 'gpt-4o-mini',
    messages,
    tools
  });
  
  const message = response.choices[0].message;
  
  // 检查是否有工具调用
  if (message.tool_calls) {
    messages.push(message);
    
    // 执行工具
    for (const toolCall of message.tool_calls) {
      const functionName = toolCall.function.name;
      const args = JSON.parse(toolCall.function.arguments);
      
      console.log(`调用: ${functionName}(${JSON.stringify(args)})`);
      
      const result = functions[functionName](args);
      
      // 添加工具结果到对话
      messages.push({
        role: 'tool',
        tool_call_id: toolCall.id,
        content: JSON.stringify(result)
      });
    }
    
    // 第二次调用：AI 使用工具结果生成回复
    const finalResponse = await openai.chat.completions.create({
      model: 'gpt-4o-mini',
      messages,
      tools
    });
    
    return finalResponse.choices[0].message.content;
  }
  
  return message.content;
}

// 使用
const reply = await chatWithTools('北京今天天气怎么样？帮我通知张三带伞');
console.log(reply);

## 3. ReAct 模式

Reasoning（推理）+ Acting（行动）的循环：

In [ ]:
// 思考 → 行动 → 观察 → 思考 → ... → 回答
async function reactAgent(userQuery) {
  const messages = [
    {
      role: 'system',
      content: `你遵循 ReAct 模式：
        1. Thought: 思考当前状态
        2. Action: 调用工具
        3. Observation: 观察结果
        4. 重复直到能回答用户`
    },
    { role: 'user', content: userQuery }
  ];
  
  // 最大迭代次数
  for (let i = 0; i < 5; i++) {
    const response = await openai.chat.completions.create({
      model: 'gpt-4o-mini',
      messages,
      tools
    });
    
    const message = response.choices[0].message;
    messages.push(message);
    
    if (!message.tool_calls) {
      return message.content;  // 最终答案
    }
    
    // 执行工具并继续
    for (const call of message.tool_calls) {
      const result = functions[call.function.name](
        JSON.parse(call.function.arguments)
      );
      messages.push({
        role: 'tool',
        tool_call_id: call.id,
        content: JSON.stringify(result)
      });
    }
  }
}

## 练习

1. 实现一个计算器 Agent（支持 + - * /）
2. 实现多步查询 Agent（查天气→查路线→规划行程）
3. 给 OpenClaw 添加一个自定义工具